In [1]:
import sacrebleu

# Define translations and references
translations = [
    "The bride's father symbolically offers to the bridegroom a cow as a present.",
    'Early pregnancy bleeding is usually from a maternal source, rather than a fetal one.'
]

references = [
    '<2hi> दुल्हन का पिता प्रतीकात्मक रूप से दूल्हे को एक गाय उपहार के रूप में देता है। </s>',
    '<2hi> प्रारंभिक गर्भावस्था रक्तस्राव आमतौर पर भ्रूण के बजाय मातृ स्रोत से होता है। </s>'
]

# Compute BLEU score
bleu = sacrebleu.corpus_bleu(translations, [references])
print(f"BLEU Score: {bleu.score:.4f}")


BLEU Score: 0.4260


In [2]:
from transformers import AutoTokenizer

# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART", do_lower_case=False, use_fast=False, keep_accents=True)

# Input and Output sequences
x = "I am a boy </s> <2en>"
y = "<2hi> मैं एक लड़का हूँ </s>"

# Tokenize inputs and outputs
inp = tokenizer(x, add_special_tokens=False, return_tensors="pt", padding=True).input_ids
out = tokenizer(y, add_special_tokens=False, return_tensors="pt", padding=True).input_ids

print("Tokenized Input:", inp)
print("Tokenized Output:", out)





/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Tokenized Input: tensor([[  466,  1981,    80, 25573, 64001, 64004]])
Tokenized Output: tensor([[64006,   942,    43, 32720,  8384, 64001]])


In [3]:
from transformers import AutoTokenizer
import torch
from src.gpt2 import GPT2Config, GPT2LMHeadModel
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


64014
64000


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (-1). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Encoder Model 
Decoder only model 

 I am a boyिऩैपटिये नीटिलोपटियेपटिये Srinagar ऱेडियो जय़न्त हूवु कोऴिक्कोट् करुछि रॆकार्ड्सदा चुके1 keyमानिपटियेlate🍿 लड़के ऱेडियो गुस्से मॆत्त छोटबेला transparencylate रक्तदाब鹿कड़ा संबंधालु midकड़ा छोटबेला बटन्ुविन्ऱॆ कत्तोलिक्क आा्सदा ऱेडियोपटिये ऱेडियो ऒरासस्
145339392
147490318


In [36]:

# Decoder-Only Model Configuration
dec_only_config = GPT2Config(
    vocab_size=64014,
    bos_token_id=64000,
    n_layer=1,
    pad_token_id=0,
    eos_token_id=64001
)
model = GPT2LMHeadModel(config=dec_only_config)

print("Decoder-Only Model Trainable Parameters:", model.num_parameters(only_trainable=True))
print("Encoder-Decoder Model Trainable Parameters:", enc_dec_model.num_parameters(only_trainable=True))

# 57038592
# 145339392

Decoder-Only Model Trainable Parameters: 57038592
Encoder-Decoder Model Trainable Parameters: 145339392


In [8]:
# Evaluation functions
def compute_bleu_score(predictions, references):
    return sacrebleu.corpus_bleu(predictions, [references]).score

def compute_chrf_score(predictions, references):
    score = sacrebleu.corpus_chrf(predictions, [references])
    return score.score

def compute_ter_score(predictions, references):
    ter = sacrebleu.corpus_ter(predictions, [references])
    return ter.score

In [10]:
outputs = model(input_ids=inp, labels=out)

In [25]:
outputs = model.generate(input_ids=inp, max_length=50, num_beams=5, early_stopping=True)

In [26]:
predictions = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs]

In [27]:
predictions

['I am a boy नागरिकांना रिजभी रिजभी रिजभीソ42424242 कुऱि🎡🎡🎡 मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् suggested suggested suggestedातमकातमक संबादमाध्यमातमकातमकातमक मऩम् मऩम् मऩम् मऩम्testestes']

In [28]:
bleu_score = compute_bleu_score(predictions, out)

In [29]:
import sacrebleu

def compute_bleu_score(predictions, references):
    # Convert tensors to lists of strings
    predictions_list = [str(pred) for pred in predictions]
    references_list = [str(ref) for ref in references]

    # Compute BLEU score
    bleu = sacrebleu.corpus_bleu(predictions_list, [references_list])
    return bleu.score

bleu_score = compute_bleu_score(predictions, out)
print(f"BLEU Score: {bleu_score}")


BLEU Score: 0.5799856917484751


In [46]:
from src.mbart.configuration_mbart import MBartConfig
from src.mbart.modeling_mbart import MBartModel, MBartForConditionalGeneration
from src.mbart.tokenization_mbart import MBartTokenizer

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import AlbertTokenizer, AutoTokenizer


from src.xlnet.modeling_xlnet import XLNetLMHeadModel
from src.xlnet.configuration_xlnet import XLNetConfig


enc_dec_config = MBartConfig(vocab_size = 64014, bos_token_id= 64000, 
                     activation_dropout=0.1, attention_dropout=0.1,encoder_layers=2,
                     decoder_layers=3,pad_token_id=0,eos_token_id=64001)
enc_dec_model = MBartForConditionalGeneration(config=enc_dec_config)

print(len(tokenizer))
print(tokenizer.vocab_size)

dec_only_config = XLNetConfig(vocab_size = 64014, bos_token_id= 64000, n_layer=6,pad_token_id=0,eos_token_id=64001)
dec_only_model = XLNetLMHeadModel(config=dec_only_config)

x = "I am a boy </s> <2en>"
y = "<2hi> मैं  एक लड़का हूँ </s>"





inp = tokenizer(x, add_special_tokens=False, return_tensors="pt", padding=True).input_ids
out = tokenizer(y, add_special_tokens=False, return_tensors="pt", padding=True).input_ids 


output_enc_dec = enc_dec_model(input_ids=inp, decoder_input_ids=out[:,0:-1], labels=out[:,1:])

output_dec = dec_only_model(input_ids=inp, labels=out)






bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")

enc_dec_out = tokenizer.decode(
    enc_dec_model.generate(inp, use_cache=True, 
                           num_beams=4, max_length=20, min_length=1, early_stopping=True, pad_token_id=pad_id, 
                           bos_token_id=bos_id, eos_token_id=eos_id,
                           decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>"))[0]
, skip_special_tokens=True, clean_up_tokenization_spaces=False)

# enc_dec_out = tokenizer.decode(enc_dec_model.generate(inp, max_length=50, num_return_sequences=1)[0], skip_special_tokens=True)

dec_only_out = dec_only_model.generate(inp, max_length=50, num_return_sequences=1)
dec_only_out  = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs]
print("Encoder Model",enc_dec_out)
print("Decoder only model \n\n",dec_only_out)



print(enc_dec_model.num_parameters(only_trainable=True))
print(dec_only_model.num_parameters(only_trainable=True))


64014
64000
Encoder Model OWOWOWOWOWOWOWOWOWOWOWOWOWOWOWOWOWOW
Decoder only model 

 ['I am a boy नागरिकांना रिजभी रिजभी रिजभीソ42424242 कुऱि🎡🎡🎡 मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् मऩम् suggested suggested suggestedातमकातमक संबादमाध्यमातमकातमकातमक मऩम् मऩम् मऩम् मऩम्testestes']
145339392
147490318


In [44]:
out

tensor([[64006,   942,    43, 32720,  8384, 64001]])

In [45]:
bleu_score = compute_bleu_score(dec_only_out, out)
print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.5799856917484751
